# 设施选址问题

## 目标和前提条件

设施选址问题在许多行业中都很常见，包括物流和电信行业。在这个例子中，我们将向您展示如何处理一个设施选址问题，该问题涉及确定供应一组超市所需的仓库数量和位置。我们将演示如何构建这个问题的混合整数规划(MIP)模型，如何在Gurobi Python API中实现这个模型，然后使用Gurobi优化器找到最优解。

这是一个初级建模示例，我们假设您了解Python并且对构建数学优化模型有一定了解。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 动机

设施选址问题的研究（也称为"位置分析"[1]）是运筹学和计算几何的一个分支，主要关注设施的最优布置以最小化运输成本，同时考虑诸如安全性（例如避免将危险材料放置在住宅区附近）和竞争对手设施位置等因素。

费马-韦伯问题是17世纪首次提出的设施选址问题之一。
费马-韦伯问题可以描述如下：在平面上给定三个点，找到第四个点，使得该点到给定三点的距离之和最小。这个问题可以看作是设施选址问题的一种变体，其中假设所有目的地的单位距离运输成本都相同。

设施选址问题在各种行业都有应用。在供应链管理和物流领域，这个问题可用于寻找商店、工厂、仓库等的最优位置。其他应用范围从公共政策（如城市中警察的部署）、电信（如网络中的基站塔）、甚至粒子物理（如排斥电荷之间的分离距离）。设施选址问题的另一个应用是确定天然气输送设备的位置。最后，设施选址问题还可以应用于聚类分析。

## 问题描述


一家英国大型连锁超市需要为其在英格兰北部开设的一系列超市建立仓库。超市的位置已经确定，但仓库的位置还需要确定。

已经确定了几个适合建设仓库的候选地点，但需要决定
开设多少个仓库以及在哪些候选地点建设仓库。

开设多个仓库会带来优势，因为这会减少卡车从仓库到超市的平均行驶距离，从而降低配送成本。但是，开设仓库会产生固定成本。

在这个例子中，我们的目标是在配送成本和建设新设施成本之间找到最优的平衡点。

## 解决方案方法

数学规划是一种声明式方法，建模者通过构建数学优化模型来捕获复杂业务问题的关键方面。Gurobi优化器使用最先进的数学和计算机科学方法求解这些模型。

数学优化模型有五个组成部分，即：

* 集合和索引
* 参数
* 决策变量
* 目标函数
* 约束条件

现在我们来介绍设施选址问题的MIP建模方法。

## 模型公式

### 集合和索引

$i \in I$: 超市（或客户）位置的索引和集合。

$j \in J$: 候选仓库（或设施）位置的索引和集合。

### 参数

$f_{j} \in \mathbb{R}^+$: 与建设设施 $j \in J$ 相关的固定成本。

$d_{i,j} \in \mathbb{R}^+$: 设施 $j \in J$ 和客户 $i \in I$ 之间的距离。

$c_{i,j} \in \mathbb{R}^+$: 候选设施位置 $j \in J$ 和客户位置 $i \in I$ 之间的运输成本。我们假设这个成本与设施和客户之间的距离成正比。即 $c_{i,j} = \alpha \cdot d_{i,j}$，其中 $\alpha$ 是每英里的运输成本，经过调整以考虑五年期间预计的平均运输次数。

### 决策变量

$select_{j} \in \{0, 1 \}$: 如果我们在候选位置 $j \in J$ 建设设施，则该变量等于1；否则等于0。

$0 \leq assign_{i,j} \leq 1$: 这个非负连续变量确定客户 $i \in I$ 从设施 $j \in J$ 接收的供应比例。

### 目标函数

- **总成本**。我们希望最小化开设和运营设施的总成本。这是开设设施的成本和设施与客户之间运输成本的总和。这个总成本衡量了建设新设施的成本和五年期间总配送成本之间的权衡。

\begin{equation}
\text{Min} \quad Z = \sum_{j \in J} f_{j} \cdot select_{j} + \sum_{j \in J} \sum_{i \in I} c_{i,j} \cdot assign_{i,j}
\tag{0}
\end{equation}

### 约束条件

- **需求**。对于每个客户 $i \in I$，确保其需求得到满足。即，每个客户从每个设施接收的供应比例之和必须等于1：

\begin{equation}
\sum_{j \in J} assign_{i,j} = 1 \quad \forall i \in I
\tag{1}
\end{equation}

- **运输**。我们需要确保只有在设施 $j \in J$ 实际建成的情况下，才能从该设施进行运输。

\begin{equation}
assign_{i,j} \leq select_{j} \quad \forall i \in I \quad \forall j \in J
\tag{2}
\end{equation}

## Python 实现

这个例子考虑了两个超市和九个仓库候选地点。下表提供了每个超市的坐标。

| <i></i> | 坐标 |  
| --- | --- | 
| 超市 1 | (0,1.5) |
| 超市 2 | (2.5,1.2) |

下表显示了候选仓库地点的坐标和建设仓库的固定成本（以百万英镑为单位）。

| <i></i> | 坐标 | 固定成本 |
| --- | --- |  --- |
| 仓库 1 | (0,0) | 3 |
| 仓库 2 | (0,1) | 2 |
| 仓库 3 | (0,2) | 3 |
| 仓库 4 | (1,0) | 1 |
| 仓库 5 | (1,1) | 3 | 
| 仓库 6 | (1,2) | 3 |
| 仓库 7 | (2,0) | 4 |
| 仓库 8 | (2,1) | 3 |  
| 仓库 9 | (2,2) | 2 |


每英里的运输成本为一百万英镑。

## Python 实现

现在我们导入Gurobi Python模块和其他Python库。然后，我们用给定的数据初始化数据结构。

In [ ]:
# %pip install gurobipy

In [1]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB

# 测试环境：Gurobi v9.1.0 和 Python 3.7.0

# 参数
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1

### 预处理

我们定义一个函数来确定每个设施和客户位置之间的欧几里得距离。此外，我们还计算了设施选址问题的MIP模型公式所需的关键参数。

In [ ]:
# 这个函数确定设施和客户位置之间的欧几里得距离。

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

# 计算MIP模型公式的关键参数

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# 计算运输成本

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

### 模型部署

现在我们确定设施选址问题的MIP模型，定义决策变量、约束条件和目标函数。接下来，我们开始优化过程，Gurobi找到最小化总成本的设施建设计划。

In [5]:
# MIP模型公式

m = gp.Model('facility_location')

select = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
assign = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='Assign')

m.addConstrs((assign[(c,f)] <= select[f] for c,f in cartesian_prod), name='Setup2ship')
m.addConstrs((gp.quicksum(assign[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

m.setObjective(select.prod(setup_cost)+assign.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Set parameter LicenseID to value 2601452
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 20 rows, 27 columns and 54 nonzeros
Model fingerprint: 0x0939f503
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Found heuristic solution: objective 25.0385165

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0     

## 分析


优化模型的结果显示，最小总成本值为472万英镑。让我们看看实现该最优结果的解决方案。

### 仓库建设计划

该计划确定在哪些地点建设仓库。

In [6]:
# 显示决策变量的最优值

for facility in select.keys():
    if (abs(select[facility].x) > 1e-6):
        print(f"\n 在位置 {facility + 1} 建设一个仓库。")


 在位置 4 建设一个仓库。


### 运输计划

该计划确定从每个建成的设施到每个客户的运输比例。

In [7]:
# 从设施到客户的运输。

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].x) > 1e-6):
        print(f"\n 超市 {customer + 1} 从仓库 {facility + 1} 接收其需求的 {round(100*assign[customer, facility].x, 2)} %。")



 超市 1 从仓库 4 接收其需求的 100.0 %。

 超市 2 从仓库 4 接收其需求的 100.0 %。


## 结论
在这个例子中，我们解决了一个设施选址问题，我们希望建设仓库以供应大量超市，同时最小化建设仓库的固定总成本和从仓库到超市的总运输成本。我们学习了如何将问题公式化为MIP模型。我们还学习了如何实现MIP模型公式并使用Gurobi Python API求解它。

## 参考文献
[1] Laporte, Gilbert, Stefan Nickel, and Saldanha da Gama, Francisco. Location Science. Springer, 2015.

版权所有 © 2020 Gurobi Optimization, LLC